> First time use: follow instructions in the README.md file in this directory.


In [1]:
import timelink
timelink.version

'1.1.18'


# Receitas / Recipes 

Várias receitas para usar Timelink com notebooks


---
 

Recipes for using Timelink with notebooks




## Setup

### Minimal

Create a `TimelinkNotebook` instance.

Default values will create a sqlite database.

First time run takes a while because auxiliary software will be downloaded


In [3]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name='tutorial2')

# to use postgresql do instead:
# tlnb = TimelinkNotebook(db_type='postgres')

### Changing the default values

When called with no parameters `TimelinkNotebook` 
assumes a certain directory layout containing the notebook
and from that layout autoconfigures various parameters.

The default layout:

* **project-directory**
    * **database** (database related filed)
    * **notebooks**  _(directory with the current notebook)_ 
    * **kleio**  (source files in kleio format)
    * **identifications** (files with identification information)

Based on this layout `TimelinkNotebook` assumes a few default values:

* **database name**
    * the name of the *project directory* is used as the **database name**, sanitized to produce a valid database name
      (hifens and spaces are replaced with _ ).
* **database type**
  * the type of the database is assumed to be **sqlite**  and a **sqlite** database is created in 
      *database/sqlite*
    * the `db_type` parameter can be used to set the database to `postgres`. 
* **Kleio server home**, working directory for the _Kleio server_ is assumed to be the **project directory**
  * The _Kleio server_ is an application
    that processes transcriptions of historical _Kleio_ notation and generates data in a format
    that can be imported to the database.

These default values can be changed with parameters when creating the `TimelinkNotebook`  instance:

```python
tlnb = TimelinkNotebook(
    project_name="my-project",
    project_home="~/projects/m-project",
    db_type="postgres",
    db_name="my_project_db",
    kleio_image="timelinkserver/kleio-server",
    kleio_version="12.0.0",
    postgres_image="postgres",
    postgres_version="latest",
    sqlite_dir="~/databases",
    sql_echo=True
)
```

### Examining the current configuration

In [4]:
tlnb.print_info()
# to see tokens and postgres password do:
# tlnb.print_info(show_token=True, show_password=True)

Timelink version: 1.1.18
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial2
Kleio image: timelinkserver/kleio-server
Kleio server token: 4ngzG...
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: exciting_noyce
Kleio version requested: latest
Kleio server version: 12.7.579 (2025-01-29 17:45:15)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial2, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=

### Listing available databases

In [5]:
pd = tlnb.get_postgres_databases()
print(f"Postgres databases: {pd}")
sd = tlnb.get_sqlite_databases()
print(f"Sqlite databases: {sd}")


Postgres databases: ['timelink', 'timelink-web', 'test_db', 'tests_users', 'test_project', 'tests']
Sqlite databases: ['../database/sqlite/tutorial.sqlite', '../database/sqlite/orm_tutorial.sqlite', '../database/sqlite/tutorial2.sqlite']


## Database status (table row count)

In [6]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,acusacoes,0
2,adendas,0
3,aforamentos,0
4,alembic_version,1
5,aregisters,0
6,attributes,15452
7,blinks,0
8,cartas,0
9,casos,0


## Dealing with kleio files

### List of running Kleio Servers

In [7]:
from timelink.kleio.kleio_server import list_kleio_server_containers


for (name,home,port,token,cont) in list_kleio_server_containers():
    labels = cont.labels
    build = labels.get("BUILD", "")
    version = labels.get("VERSION", "")
    build_date = labels.get("BUILD_DATE", "")
    print(f"Kleio server: {name:20}  {port} {home} {token} v{version}.{build} {build_date}")


Kleio server: exciting_noyce        8089 /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial 4ngzG9IHHeqd7EAHFhVx7i06howNP2e7 v12.7.579 2025-01-29 17:45:15
Kleio server: compassionate_ellis   8093 /Users/jrc/develop/timelink-py/tests/timelink-home Z38HoRV7IC8tZiykWosHgPs1VOqEeO0e v12.7.579 2025-01-29 17:45:15
Kleio server: tender_noether        8091 /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project J26sSjctZZHj9Y2e9Cwwni1HUUewVRya v12.7.579 2025-01-29 17:45:15
Kleio server: nostalgic_hawking     8092 /Users/jrc/mhk-home/sources/demo_sources OBsky2ATFqPpRCDIxowtTtkQfNOcf9nq v12.7.579 2025-01-29 17:45:15
Kleio server: awesome_joliot        8088 /Users/jrc/develop/timelink-py/tests/timelink-home/projects/web-tests lbK1eg2jFT6OLfFWf6Ha6V2moxSKwXXU v12.7.579 2025-01-29 17:45:15
Kleio server: awesome_ramanujan     8090 /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project/sources/reference_sources/rentities 2rbmVn1P8MtSZ3XEYDdQsL5psN

### List Kleio files available

In [7]:
kleio_files = tlnb.get_kleio_files()
kleio_files.head()

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/auc-alunos.cli,auc-alunos.cli,2025-01-10 08:09:08.127463+00:00,V,2025-01-10 08:09:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-10 08:25:44.062796,/rest/reports/sources/auc-alunos.rpt,/rest/exports/sources/auc-alunos.xml
1,sources/b1685.cli,b1685.cli,2025-01-10 07:07:03.744927+00:00,W,2025-01-10 07:07:00+00:00,0,1,I,0,0,No errors,No warnings,2025-01-10 07:40:09.603337,/rest/reports/sources/b1685.rpt,/rest/exports/sources/b1685.xml
2,sources/dehergne-a.cli,dehergne-a.cli,2025-01-10 07:07:07.206591+00:00,V,2025-01-10 07:07:00+00:00,0,0,E,3,0,ERROR: dehergne-a.cli processing same_as relat...,No warnings,2025-01-10 07:39:25.763429,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
3,sources/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2025-01-10 07:07:05.549587+00:00,V,2025-01-10 07:07:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-10 07:39:42.862503,/rest/reports/sources/dehergne-locations-1644.rpt,/rest/exports/sources/dehergne-locations-1644.xml
4,sources/real-entities/real-entities.cli,real-entities.cli,2025-01-10 07:07:05.796856+00:00,V,2025-01-10 07:07:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-10 07:39:53.794528,/rest/reports/sources/real-entities/real-entit...,/rest/exports/sources/real-entities/real-entit...


Show only translation and import status

In [8]:
kleio_files[["name","status","import_status"]]

,name,status,import_status
0,auc-alunos.cli,V,I
1,b1685.cli,W,I
2,dehergne-a.cli,V,E
3,dehergne-locations-1644.cli,V,I
4,real-entities.cli,V,I


Show translation and import errors

In [9]:
kleio_files[["name","errors","warnings","import_errors","import_warnings"]]

,name,errors,warnings,import_errors,import_warnings
0,auc-alunos.cli,0,0,0,0
1,b1685.cli,0,1,0,0
2,dehergne-a.cli,0,0,3,0
3,dehergne-locations-1644.cli,0,0,0,0
4,real-entities.cli,0,0,0,0


### Translation and Import reports

Translation reports.

Pass a dataframe with Kleio files and a row number to get the translation report

In [10]:
rpt=tlnb.get_translation_report(kleio_files,rows=[1])
print(rpt)

KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
10-1-2025 7-7

Processing data file b1685.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 41
Obs: 
** Processing source fonte$baptismos 1685

Near lines: 3 
Near lines: 4       bap$b1685.1/8/7/1685/?/manuel cordeiro

6: bap$b1685.1
22: bap$b1685.2
40: bap$b1685.3
59: bap$b1685.4
70: bap$b1685.5
79: bap$b1685.6
98: bap$b1685.7
113: bap$b1685.8
129: bap$b1685.9
144: bap$b1685.10
156: bap$b1685.11
171: bap$b1685.12
183: bap$b1685.13
201: bap$b1685.14
219: bap$b1685.15
235: bap$b1685.16
250: bap$b1685.17
266: bap$b1685.18
284: bap$b1685.19
301: bap$b1685.20
318: bap$b1685.21
334: bap$b1685.22
351: bap$b1685.23
368: bap$b1685.24
381: bap$b1685.25
396: bap$b1685.26
416: bap$b1685.27
431: bap$b1685.27b
452: bap$b168

Import report

In [11]:
rpt = tlnb.get_import_rpt(kleio_files,rows=[2,3])
print(rpt)

dehergne-a.cli
ERROR: dehergne-a.cli processing same_as relation deh-luis-de-almeida-rela44: OccurrenceMissingError: Error, deh-luis-de-almeida-ref2 and deh-belchior-miguel-carneiro-leitao must exist in the database

ERROR: dehergne-a.cli processing same_as relation deh-jean-joseph-marie-amiot-rela66: OccurrenceMissingError: Error, deh-jean-joseph-marie-amiot-ref1 and deh-jean-regis-lieou must exist in the database

ERROR: dehergne-a.cli processing same_as relation deh-pierre-amys-rela76: OccurrenceMissingError: Error, deh-pierre-amys-ref1 and deh-alessandro-cicero must exist in the database
No warnings

dehergne-locations-1644.cli
No errors
No warnings





# Atualizar base de dados

Atualiza traduções de fontes e importa quando tradução não tem erros.

---

# Update database

Updates source translations and imports into database sources with no errors.

In [12]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

### Clear the translations

In [13]:
print("remove comment to clear translations")
# tlnb.kleio_server.translation_clean(path="",recurse="yes")

remove comment to clear translations


Show imported files status

In [14]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                5 non-null      object             
 1   name                5 non-null      object             
 2   size                5 non-null      int64              
 3   directory           5 non-null      object             
 4   modified            5 non-null      datetime64[ns, UTC]
 5   modified_iso        5 non-null      datetime64[ns, UTC]
 6   modified_string     5 non-null      object             
 7   qtime               5 non-null      datetime64[ns, UTC]
 8   qtime_string        5 non-null      object             
 9   source_url          5 non-null      object             
 10  status              5 non-null      object             
 11  translated          5 non-null      datetime64[ns, UTC]
 12  translated_string   5 non-null      obje

Check the import status of the translated files
```python
    I = "I" # imported
    E = "E" # imported with error
    W = "W" # imported with warnings no errors
    N = "N" # not imported
    U = "U" # translation updated need to reimport

``` 

In [15]:
imported_files_df[["import_status","import_errors","import_warnings","name","imported","path"]].sort_values("name")

,import_status,import_errors,import_warnings,name,imported,path
0,I,0,0,auc-alunos.cli,2025-01-10 08:25:44.062796,sources/auc-alunos.cli
1,I,0,0,b1685.cli,2025-01-10 07:40:09.603337,sources/b1685.cli
2,E,3,0,dehergne-a.cli,2025-01-10 07:39:25.763429,sources/dehergne-a.cli
3,I,0,0,dehergne-locations-1644.cli,2025-01-10 07:39:42.862503,sources/dehergne-locations-1644.cli
4,I,0,0,real-entities.cli,2025-01-10 07:39:53.794528,sources/real-entities/real-entities.cli


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,rows=List[int],status="T"])
* TimelinkNotebook.import(files_df, rows=List[int])



# Explorar a base de dados
---

# Exploring the database

## Database status

Number of rows of each table in the database

In [16]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,alembic_version,1
2,aregisters,0
3,attributes,15452
4,blinks,0
5,class_attributes,63
6,classes,13
7,entities,18609
8,geoentities,215
9,goods,0


## List of tables available

In [17]:
tlnb.db.table_names()

['acts',
 'alembic_version',
 'aregisters',
 'attributes',
 'blinks',
 'class_attributes',
 'classes',
 'entities',
 'geoentities',
 'goods',
 'kleiofiles',
 'links',
 'objects',
 'persons',
 'relations',
 'rentities',
 'rgeoentities',
 'robjects',
 'rpersons',
 'sources',
 'syslog',
 'syspar']

## List of models available


In [18]:
tlnb.db.get_models_ids()

['attribute',
 'relation',
 'act',
 'source',
 'aregister',
 'person',
 'good',
 'object',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'class',
 'entity']

### Get a model class by name

The model class is used to build queries (see later receipts on queries)

In [19]:
person_model = tlnb.db.get_model("person")
print(person_model)

<class 'timelink.api.models.person.Person'>


## Describe a table

In [20]:
tlnb.db.describe("persons",show=True)
print()
tlnb.db.describe("entities",show=True)


persons
id                   persons              VARCHAR    {ForeignKey('entities.id')}
name                 persons              VARCHAR    
sex                  persons              VARCHAR(1) 
obs                  persons              VARCHAR    

entities
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_source', String(), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('extra_info', JSON(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0xffffa9dbc820>)),
 Column('indexed', DateTime(), table=<entities>)]

### Describe a model

A Model includes all the information in the table hierarchy.

For instance, the model Person includes the information in the table "persons"
but also the information in the table "entities" related to a specific person

In [21]:
person_model = tlnb.db.get_model("person")
tlnb.db.describe(person_model, show=True)

<class 'timelink.api.models.person.Person'>
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   persons              VARCHAR    {ForeignKey('entities.id')}
name                 persons              VARCHAR    
sex                  persons              VARCHAR(1) 
obs                  persons              VARCHAR    


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_source', String(), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('extra_info', JSON(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0xffffa9dbc820>)),
 Column('indexed', DateTime(), table=<entities>),
 Column('id', String(), ForeignKey('entities.id'), table=<persons>, primary_key=True, nullable=False),
 Column('name', String(), table=<persons>),
 Column('sex', String(length=1), table=<persons>),
 Column('obs', String(), table=<persons>)]

It is also possibel to import directly the model from
the Timelink package

In [22]:
from timelink.api.models import Person

tlnb.db.describe(Person,show=True)

<class 'timelink.api.models.person.Person'>
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   persons              VARCHAR    {ForeignKey('entities.id')}
name                 persons              VARCHAR    
sex                  persons              VARCHAR(1) 
obs                  persons              VARCHAR    


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_source', String(), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('extra_info', JSON(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0xffffa9dbc820>)),
 Column('indexed', DateTime(), table=<entities>),
 Column('id', String(), ForeignKey('entities.id'), table=<persons>, primary_key=True, nullable=False),
 Column('name', String(), table=<persons>),
 Column('sex', String(length=1), table=<persons>),
 Column('obs', String(), table=<persons>)]

# Searching the database

### Procurar pessoa

---
### Search for people

#### Search persons by name

In [23]:
from timelink.api.models import Person
from sqlalchemy import select

show_only=30

with tlnb.db.session() as session:
    stmt = select(Person).where(Person.name.like('%vasconcelos')).order_by('name')
    print(stmt)
    persons = session.execute(stmt).scalars().all()
    print()
    for person in persons[:show_only]:
        print(person.id, person.name,person.sex)
        print(person.to_kleio())

SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_source, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.extra_info, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.name LIKE :name_1 ORDER BY persons.name

b1685.6-per4 joao da costa de vasconcelos m
pad$joao da costa de vasconcelos/m/id=b1685.6-per4
  rel$function-in-act/pad/bap/b1685.6/1685-08-27/obs=line: 88
  ls$residencia/soure/1685-08-27
b1685.27b-per3 joao da costa de vasconcelos m
pad$joao da costa de vasconcelos/m/id=b1685.27b-per3
  rel$function-in-act/pad/bap/b1685.27b/1685-11-25/obs=line: 445
b1685.29-per4 joao da costa de vasconcelos m
pad$joao da costa de vasconcelos/m/id=b1685.29-per4
  rel$function-in-act/pad/bap/b1685.29/1685-12-13/obs=line: 511


In [24]:
[person.id for person in persons]

['b1685.6-per4', 'b1685.27b-per3', 'b1685.29-per4']

### Search other Entities

#### get the Entity classes in the database

In [25]:
from sqlalchemy import select, func
from timelink.api.models import Entity

models = tlnb.db.get_models_ids()
models

['attribute',
 'relation',
 'act',
 'source',
 'aregister',
 'person',
 'good',
 'object',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'class',
 'entity']

#### Get columns of an entity type

In [26]:
table = tlnb.db.get_table("entity")
print(table.name)
list(table.columns)

entities


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_source', String(), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('extra_info', JSON(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0xffffa9dbc820>)),
 Column('indexed', DateTime(), table=<entities>)]

#### Search any entity type


In [27]:
from timelink.api.models import Entity
from sqlalchemy import select, func

Geoentity = tlnb.db.get_model("geoentity")
tlnb.db.describe(Geoentity,show=True)
stmt = select(Geoentity).where(Geoentity.name.like('Hang%'))
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.the_type,row.name,row.id,row.obs)
        print(row.to_kleio())

<class 'timelink.api.models.geoentity.Geoentity'>
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   geoentities          VARCHAR    {ForeignKey('entities.id')}
name                 geoentities          VARCHAR    
the_type             geoentities          VARCHAR(32) 
obs                  geoentities          VARCHAR    
SELECT geoentities.id, entities.id AS id_1, entities.class, entities.inside, entities

### Search Acts by date

In [28]:
from timelink.api.models import Act
from sqlalchemy import select, func

tlnb.db.describe(Act,show=True)

stmt = select(Act).where(Act.the_date > '16850800',
                         Act.the_date < '16850900')

print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.groupname,row.the_date,row.loc,row.obs)
        print(row.to_kleio())

<class 'timelink.api.models.act.Act'>
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   acts                 VARCHAR    {ForeignKey('entities.id')}
the_type             acts                 VARCHAR(32) 
the_date             acts                 VARCHAR    
loc                  acts                 VARCHAR    
ref                  acts                 VARCHAR    
obs                  acts                 V

### Search relations

In [29]:
from timelink.api.models import Relation
from sqlalchemy import select

tlnb.db.describe(Relation,show=True)

stmt = select(Relation).where(Relation.destination=='b1685.8-per1')
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.origin,row.the_type, row.the_value,row.destination)
        print(row.to_kleio())

<class 'timelink.api.models.relation.Relation'>
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   relations            VARCHAR    {ForeignKey('entities.id')}
origin               relations            VARCHAR    {ForeignKey('entities.id')}
destination          relations            VARCHAR    {ForeignKey('entities.id')}
the_type             relations            VARCHAR    
the_value            relations    

#### Get person by id

Show a single person or entity in Kleio notation

In [30]:
p = tlnb.db.get_person(person.id)
print(p.to_kleio())

pad$joao da costa de vasconcelos/m/id=b1685.29-per4
  rel$function-in-act/pad/bap/b1685.29/1685-12-13/obs=line: 511


### Show other type of entities by id in Kleio

In [31]:
from timelink.api.models import Entity

with tlnb.db.session() as session:
    ent = tlnb.db.get_entity("deh-r1644-chekiang",session)
    print(ent.to_kleio())

geo1$Chekiang#Tche-kiang, hoje:Zhejiang, 浙江, @wikidata:Q16967/geo1
  rel$function-in-act/geo1/geodesc/deh-chre-1644/1644/obs=line: 99
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q16967"#Tche-kiang, hoje:Zhejiang, 浙江, @wikidata:Q16967%Q16967/1644




###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [32]:
from sqlalchemy import inspect

inspect(tlnb.db.engine).get_view_names()

['eattributes', 'nattributes', 'nfunctions']

In [33]:
eview = tlnb.db.create_eattribute_view()
tlnb.db.describe(eview,show=True)

eattributes
id                   eattributes          VARCHAR    
the_line             eattributes          INTEGER    
the_level            eattributes          INTEGER    
the_order            eattributes          INTEGER    
groupname            eattributes          VARCHAR    
updated              eattributes          DATETIME   
indexed              eattributes          DATETIME   
e_extra_info         eattributes          JSON       
entity               eattributes          VARCHAR    {ForeignKey('entities.id')}
the_type             eattributes          VARCHAR    
the_value            eattributes          VARCHAR    
the_date             eattributes          VARCHAR    
aobs                 eattributes          VARCHAR    
a_extra_info         eattributes          JSON       


[Column('id', String(), table=<eattributes>, primary_key=True, nullable=False),
 Column('the_line', Integer(), table=<eattributes>),
 Column('the_level', Integer(), table=<eattributes>),
 Column('the_order', Integer(), table=<eattributes>),
 Column('groupname', String(), table=<eattributes>),
 Column('updated', DateTime(), table=<eattributes>),
 Column('indexed', DateTime(), table=<eattributes>),
 Column('e_extra_info', JSON(), table=<eattributes>),
 Column('entity', String(), ForeignKey('entities.id'), table=<eattributes>),
 Column('the_type', String(), table=<eattributes>),
 Column('the_value', String(), table=<eattributes>),
 Column('the_date', String(), table=<eattributes>),
 Column('aobs', String(), table=<eattributes>),
 Column('a_extra_info', JSON(), table=<eattributes>)]

In [34]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='soure',
                    show_elements=['name','sex'],
                    name_like='%',
                    more_attributes=['profissao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 140554 to 140533
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     4 non-null      object
 1   sex                      4 non-null      object
 2   naturalidade.type        4 non-null      object
 3   naturalidade             4 non-null      object
 4   naturalidade.date        4 non-null      object
 5   naturalidade.line        4 non-null      int64 
 6   naturalidade.level       4 non-null      int64 
 7   naturalidade.obs         4 non-null      object
 8   naturalidade.extra_info  4 non-null      object
 9   profissao                0 non-null      object
dtypes: int64(2), object(8)
memory usage: 352.0+ bytes


In [65]:
df.head(5)

,name,sex,residencia.type,residencia,residencia.date,residencia.line,residencia.level,residencia.obs,residencia.extra_info,profissao,profissao.date,profissao.obs,profissao.extra_info
id,,,,,,,,,,,,,
b1685.27d-per1-per2,antonio francisco,m,residencia,soure,16850430,469,4,,{},NaN,NaN,NaN,NaN
b1685.27d-r1,antonio da rocha,m,residencia,soure,16850430,474,4,,{},NaN,NaN,NaN,NaN
b1685.3-per6,nomes desconhecido,m,residencia,soure,16850802,54,5,,{},boticario,16850802,,{}
b1685.6-per4,joao da costa de vasconcelos,m,residencia,soure,16850827,88,4,,{},NaN,NaN,NaN,NaN
b1685.10-per1-per2,matias de carvalho,m,residencia,soure,16850912,147,4,,{},NaN,NaN,NaN,NaN


#### obter attributos de outras entidades

In [35]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='geoentity',
                    show_elements=['name'],
                    the_type='activa',
                    the_value='sim',
                    more_attributes=['residencia-missao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-lampacao to deh-r1644-yunnan
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   name                          165 non-null    object
 1   activa.type                   165 non-null    object
 2   activa                        165 non-null    object
 3   activa.date                   165 non-null    object
 4   activa.line                   165 non-null    int64 
 5   activa.level                  165 non-null    int64 
 6   activa.obs                    165 non-null    object
 7   activa.extra_info             165 non-null    object
 8   activa.date.comment           26 non-null     object
 9   activa.date.original          5 non-null      object
 10  residencia-missao             38 non-null     object
 11  residencia-missao.date        38 non-null     object
 12  residencia-missao.obs         38 non-null     object


In [31]:
df.head(30)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-chala,Chala,sim,1610,None,NaN,NaN,NaN
deh-r1644-changchow-fou,Changchow,sim,1643,None,NaN,NaN,NaN
deh-r1644-changshu,Changshu,sim,1623,None,Jesuíta,1635,"R 1635 «Cham Xo», cf AHSI 28 (1951) 311-312"
deh-r1644-chengting,Chengting,sim,1621,None,NaN,NaN,NaN




###  Remover colunas sem valores

---



###  Remove empty columns

In [32]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [25]:
df.head(5)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [8]:
# create a column with the index values which are the id numbers
# Get list of people with with a certain value in a specific attribute
from timelink.pandas import entities_with_attribute
import pandas as pd

df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='Coimbra',
                    show_elements=['name','sex'],
                    more_attributes=['faculdade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()
df['id'] = df.index.values

col = 'faculdade' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc.entrada':'min',
                                                  'uc.saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 140349 to 140367
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     14 non-null     object
 1   sex                      14 non-null     object
 2   naturalidade.type        14 non-null     object
 3   naturalidade             14 non-null     object
 4   naturalidade.date        14 non-null     object
 5   naturalidade.line        14 non-null     int64 
 6   naturalidade.level       14 non-null     int64 
 7   naturalidade.obs         14 non-null     object
 8   naturalidade.extra_info  14 non-null     object
 9   faculdade                14 non-null     object
 10  faculdade.date           14 non-null     object
 11  faculdade.obs            14 non-null     object
 12  faculdade.extra_info     14 non-null     object
 13  uc.entrada               14 non-null     object
 14  uc.entrada.date          14 non-null    

,id,uc.entrada,uc.saida
faculdade,,,
Cânones,7,0000-00-00,1767-07-27
Medicina,5,1670-10-01,1823-10-20
Direito,1,1748-10-19,1748-10-19
Matemática,1,1868-10-02,1872-10-03



### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [9]:
from timelink.pandas import attribute_values

df_totals = attribute_values('grau',db=tlnb.db)


In [35]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


#### Filtrar por datas
---

#### Filter by dates



In [38]:
df_totals = attribute_values('grau',dates_between=('1772','1919'),db=tlnb.db)

In [39]:
df_totals.head(10)

,count,date_min,date_max
value,,,
Bacharel,53,1775-06-10,1912-08-03
Formatura,42,1776-05-11,1905-06-19
Licenciado,4,1800-07-26,1886-02-27
Doutor,3,1800-07-31,1887-11-27


## Visualizar entidades

---

## View entities





### Atributos de uma entidade numa tabela, uma linha por attributo

---

### Entitiy attributes in a dataframe, one line per attribute

#### A specifc entity

Use `filter_by` to filter a list of entites

In [42]:
# Get list of people with with a certain value in a specific attribute
from timelink.pandas import entities_with_attribute

df = entities_with_attribute(
                    entity_type='person',
                    the_type='uc.entrada',  # we need a base attribute
                    show_elements=['name','sex'],
                    more_attributes=['instituta','matricula.ano'],
                    filter_by=['140349'],
                    db=tlnb.db,
                    sql_echo=False)
df.sort_values('matricula.ano.date')

,name,sex,uc.entrada,uc.entrada.date,uc.entrada.obs,instituta,matricula.ano,matricula.ano.date,matricula.ano.obs
id,,,,,,,,,
140349,António de Aboim,m,1566-12-20,1566-12-20,None,None,Cânones.1571,1571-07-20,20.07.1571
140349,António de Aboim,m,1566-12-20,1566-12-20,None,None,Cânones.1573,1573-10-07,07.10.1573
140349,António de Aboim,m,1566-12-20,1566-12-20,None,None,Cânones.1574,1574-07-24,24.07.1574


In [43]:
from timelink.pandas import entities_with_attribute

neighbors = entities_with_attribute(
                                entity_type="person",
                                show_elements=["groupname","name","sex"],
                                the_type='residencia',
                                column_name="residencia",
                                the_value="soure",
                                more_attributes=['profissao'],
                                db=tlnb.db)
neighbors.head(10)

,groupname,name,sex,residencia,residencia.date,residencia.obs,profissao,profissao.date,profissao.obs
id,,,,,,,,,
b1685.27d-per1-per2,pai,antonio francisco,m,soure,16850430,None,NaN,NaN,NaN
b1685.27d-r1,pad,antonio da rocha,m,soure,16850430,None,NaN,NaN,NaN
b1685.3-per6,mrmad,nomes desconhecido,m,soure,16850802,None,boticario,16850802,None
b1685.6-per4,pad,joao da costa de vasconcelos,m,soure,16850827,None,NaN,NaN,NaN
b1685.10-per1-per2,pai,matias de carvalho,m,soure,16850912,None,NaN,NaN,NaN
b1685.14-per4,pad,manuel duarte de morais,m,soure,16850923,None,padre,16850923,None
b1685.14-per5,mad,maria rosado de carvalho,f,soure,16850923,None,NaN,NaN,NaN
b1685.21-per6,mrmad,manuel de tavora,m,soure,16851104,None,NaN,NaN,NaN
b1685.27c-per1-per2,pai,manuel rodrigues,m,soure,16851125,None,NaN,NaN,NaN


### Visualizar um grupo de entidades

---

#### Show the attributes of a group of entites

In [44]:
from timelink.api.models import Person
from sqlalchemy import select

select(Person).selected_columns.keys()

['id',
 'id_1',
 'class',
 'inside',
 'the_order',
 'the_level',
 'the_line',
 'groupname',
 'updated',
 'indexed',
 'name',
 'sex',
 'obs']

In [46]:
from timelink.pandas.entities_with_attribute import entities_with_attribute
from timelink.pandas.group_attributes import group_attributes

# Get list of people with with a certain value in a specific attribute
colleagues_df = entities_with_attribute(entity_type='person',
                        the_type='uc.entrada.ano',
                        the_value='1750',db=tlnb.db)

colleagues = colleagues_df.index.values
print(colleagues)

# show their attributes
colleagues_cv = group_attributes(colleagues,
    entity_type='person',
    show_elements=['name'],
    include_attributes=['instituta','matricula','faculdade','grau','exame'],
    exclude_attributes=['data-do-registo'],
    sql_echo=True,
    db=tlnb.db)

colleagues_cv.sort_values(['the_date','the_type','the_value'])

['140387' '140442' '140449' '140488' '140568' '140652' '140461' '140477'
 '140589']
SELECT persons.id, persons.name, attributes.the_type, attributes.the_value, attributes.the_date, attributes.obs AS attr_obs 
FROM entities JOIN persons ON entities.id = persons.id LEFT OUTER JOIN attributes ON attributes.entity = persons.id 
WHERE persons.id IN (__[POSTCOMPILE_id_1]) AND attributes.the_type IN (__[POSTCOMPILE_the_type_1]) AND (attributes.the_type NOT IN (__[POSTCOMPILE_the_type_2]))


,name,the_type,the_value,the_date,attr_obs
id,,,,,
140387,António Lobo da Costa Borges e Abranches,faculdade,Cânones,1750-10-01,None
140449,Luís António de Abranches,faculdade,Cânones,1750-10-01,None
140488,José Ribeiro de Abrantes,faculdade,Cânones,1750-10-01,None
140568,António da Costa e Abreu,faculdade,Cânones,1750-10-01,None
140652,Caetano de Abreu,faculdade,Leis,1750-10-01,None
140442,José Nunes Abranches,faculdade,Medicina,1750-10-01,None
140387,António Lobo da Costa Borges e Abranches,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01
140449,Luís António de Abranches,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01
140488,José Ribeiro de Abrantes,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01


In [4]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_type='postgres', db_name="tests")
tlnb.print_info()

APIError: 500 Server Error for http+docker://localhost/v1.43/containers/d512c88d57256670292320f6094c0f98c78c243fd5519d5a069e13d10d1d900f/start: Internal Server Error ("driver failed programming external connectivity on endpoint focused_turing (e0441959c5511e48888395ccaa55d051e4faaa9e3ed6ef72c51bfe75ebd28109): Bind for 0.0.0.0:8088 failed: port is already allocated")

In [48]:
from timelink.pandas.group_attributes import display_group_attributes
import pandas as pd

display_group_attributes(
    colleagues,
    entity_type='person',
    header_elements=['name'],
    header_attributes=['uc.entrada'],
    include_attributes=['instituta','matricula','faculdade','grau','exame'],
    sort_attributes=['the_date','the_type','the_value'],
    cmap_name='Pastel1',
    db=tlnb.db)

,name,uc.entrada,uc.entrada.date,uc.entrada.obs,id
0,António Lobo da Costa Borges e Abranches,1750-10-01,1750-10-01,None,140387
1,José Nunes Abranches,1750-10-01,1750-10-01,None,140442
2,Luís António de Abranches,1750-10-01,1750-10-01,None,140449
3,José Ribeiro de Abrantes,1750-10-01,1750-10-01,None,140488
4,António da Costa e Abreu,1750-10-01,1750-10-01,None,140568
5,Caetano de Abreu,1750-10-01,1750-10-01,None,140652
6,Manuel Inácio de Figueiredo Abranches,1750-10-27,1750-10-27,None,140461
7,Bartolomeu de Abrantes,1750-11-01,1750-11-01,None,140477
8,António José Gomes de Abreu,1750-12-16,1750-12-16,None,140589


,name,the_type,the_value,the_date,attr_obs,id
0,António Lobo da Costa Borges e Abranches,faculdade,Cânones,1750-10-01,None,140387
1,Luís António de Abranches,faculdade,Cânones,1750-10-01,None,140449
2,José Ribeiro de Abrantes,faculdade,Cânones,1750-10-01,None,140488
3,António da Costa e Abreu,faculdade,Cânones,1750-10-01,None,140568
4,Caetano de Abreu,faculdade,Leis,1750-10-01,None,140652
5,José Nunes Abranches,faculdade,Medicina,1750-10-01,None,140442
6,António Lobo da Costa Borges e Abranches,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01,140387
7,Luís António de Abranches,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01,140449
8,José Ribeiro de Abrantes,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01,140488
9,António da Costa e Abreu,instituta,1750-10-01,1750-10-01,01.10.1750 1750-10-01,140568


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [49]:
from timelink.mhk.models.person import Person

with tlnb.db.session() as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$António Pinto Abadeço/m/id=140337/obs="""
      """

                  Id: 140337
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000001

                  Nome        : António Pinto Abadeço
                  Data inicial: 1705-10-01
                  Data final  : 1710-10-01
                  Filiação: António Pinto

                  Naturalidade: Abrantes
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1705
                  01.10.1706
                  01.01.1707
                  01.10.1708
                  01.10.1709
                  01.10.1710
                  Instituta:
                  Bacharel: 28.06.1709
                  Formatura:
                  Licenciado:
                  Doutor:

                  Outras informações:
              """
  """
  rel$function-in-act/n/António Pinto Abadeço/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000001""/2021-05-17
  atr$data-